# Aleket Faster R-CNN training notebook

In [ ]:
%pip install pillow
%pip install numpy<2.0
%pip install torch torchvision --index-url https://download.pytorch.org/whl/cu124
%pip install matplotlib
%pip install gdown
%pip install tqdm

from IPython.display import clear_output

clear_output(wait=False)

print("ALL DEPENDENCIES INSTALLED")

In [1]:
# IMPORTS

# Standard Library
import random

# Third-Party Libraries
import numpy as np
from matplotlib import pyplot as plt
# Torch
import torch

# Utils
from aleket_dataset import AleketDataset, download_dataset
from utils import split_dataset
from training_and_evaluation import train,parse_params,TrainParams
from utils import get_model
from dataset_statisics import visualize_example_bboxes

In [ ]:
# Device Selection
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Random Seed for Reproducibility
SEED = 1
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
np_generator = np.random.default_rng(SEED)

# Dataset split
DATASET_FRACTION = 0.1
VALIDATION_FRACTION = 0.2
dataset = AleketDataset(download_dataset("dataset_patched", ""), img_size=1024)
train_set, val_set = split_dataset(dataset, DATASET_FRACTION, VALIDATION_FRACTION, np_generator)

def augment_example(dataset):
    examples = visualize_example_bboxes(dataset, image_ids_to_visualize=range(4))
    fig=plt.figure(figsize=(15, 3))
    columns = 4
    rows = 1
    for i in range(1, columns*rows +1):
        fig.add_subplot(rows, columns, i)
        plt.imshow(examples[i-1])
    plt.show()

Using device: cuda
Dataset loaded from dataset_patched


In [3]:

model = get_model(device)
params = TrainParams()
params.load("params1.json")
train(model, dataset, params, device, checkpoints=True)



Epoch 1/100; Learning rate: 0.01


Evaluating batches: 100%|██████████| 9/9 [00:03<00:00,  2.63it/s]


Time: 0:00:25; Epoch 1 Summary: 
	Train Mean Loss: 0.8326
	Validation AP@.50:.05:.95: 0.197
	Validation AP@.5: 0.496
	Validation AP@.75: 0.088
	Validation Recall@.50:.05:.95: 0.383
	Validation Recall@.5: 0.813
	Validation Recall@.75: 0.290
	Validation ACD: 1.552
	Validation AAD: 1.593
	New Best Validation AP@.50:.05:.95: 0.197

Epoch 2/100; Learning rate: 0.01


Evaluating batches: 100%|██████████| 9/9 [00:03<00:00,  2.63it/s]


Time: 0:00:23; Epoch 2 Summary: 
	Train Mean Loss: 0.5595
	Validation AP@.50:.05:.95: 0.189
	Validation AP@.5: 0.470
	Validation AP@.75: 0.101
	Validation Recall@.50:.05:.95: 0.396
	Validation Recall@.5: 0.819
	Validation Recall@.75: 0.313
	Validation ACD: 1.359
	Validation AAD: 1.507
	Best Validation AP@.50:.05:.95: 0.197

Epoch 3/100; Learning rate: 0.01


...

Time: 0:00:24; Epoch 98 Summary: 
	Train Mean Loss: 0.0465
	Validation AP@.50:.05:.95: 0.241
	Validation AP@.5: 0.571
	Validation AP@.75: 0.165
	Validation Recall@.50:.05:.95: 0.365
	Validation Recall@.5: 0.714
	Validation Recall@.75: 0.331
	Validation ACD: 0.635
	Validation AAD: 0.591
	Best Validation AP@.50:.05:.95: 0.262

Epoch 99/100; Learning rate: 0.0001


Evaluating batches: 100%|██████████| 9/9 [00:03<00:00,  2.56it/s]


Time: 0:00:24; Epoch 99 Summary: 
	Train Mean Loss: 0.0456
	Validation AP@.50:.05:.95: 0.240
	Validation AP@.5: 0.568
	Validation AP@.75: 0.163
	Validation Recall@.50:.05:.95: 0.363
	Validation Recall@.5: 0.712
	Validation Recall@.75: 0.326
	Validation ACD: 0.632
	Validation AAD: 0.592
	Best Validation AP@.50:.05:.95: 0.262

Epoch 100/100; Learning rate: 0.0001


Evaluating batches: 100%|██████████| 9/9 [00:03<00:00,  2.60it/s]


Time: 0:00:24; Epoch 100 Summary: 
	Train Mean Loss: 0.0467
	Validation AP@.50:.05:.95: 0.242
	Validation AP@.5: 0.571
	Validation AP@.75: 0.164
	Validation Recall@.50:.05:.95: 0.365
	Validation Recall@.5: 0.715
	Validation Recall@.75: 0.327
	Validation ACD: 0.633
	Validation AAD: 0.592
	Best Validation AP@.50:.05:.95: 0.262


In [4]:
model = get_model(device)
params = TrainParams()
params.load("params2.json")
train(model, dataset, params, device, checkpoints=True)


Epoch 1/100; Learning rate: 0.005


Evaluating batches: 100%|██████████| 9/9 [00:03<00:00,  2.54it/s]


Time: 0:00:25; Epoch 1 Summary: 
	Train Mean Loss: 0.8587
	Validation AP@.50:.05:.95: 0.204
	Validation AP@.5: 0.497
	Validation AP@.75: 0.121
	Validation Recall@.50:.05:.95: 0.409
	Validation Recall@.5: 0.843
	Validation Recall@.75: 0.315
	Validation ACD: 1.742
	Validation AAD: 1.771
	New Best Validation AP@.50:.05:.95: 0.204

Epoch 2/100; Learning rate: 0.005


Evaluating batches: 100%|██████████| 9/9 [00:03<00:00,  2.56it/s]


Time: 0:00:24; Epoch 2 Summary: 
	Train Mean Loss: 0.5541
	Validation AP@.50:.05:.95: 0.240
	Validation AP@.5: 0.550
	Validation AP@.75: 0.157
	Validation Recall@.50:.05:.95: 0.445
	Validation Recall@.5: 0.891
	Validation Recall@.75: 0.376
	Validation ACD: 1.660
	Validation AAD: 1.747
	New Best Validation AP@.50:.05:.95: 0.240

Epoch 3/100; Learning rate: 0.005


...

Time: 0:00:24; Epoch 98 Summary: 
	Train Mean Loss: 0.0640
	Validation AP@.50:.05:.95: 0.239
	Validation AP@.5: 0.540
	Validation AP@.75: 0.163
	Validation Recall@.50:.05:.95: 0.355
	Validation Recall@.5: 0.684
	Validation Recall@.75: 0.313
	Validation ACD: 0.696
	Validation AAD: 0.663
	Best Validation AP@.50:.05:.95: 0.267

Epoch 99/100; Learning rate: 5e-05


Evaluating batches: 100%|██████████| 9/9 [00:03<00:00,  2.56it/s]


Time: 0:00:24; Epoch 99 Summary: 
	Train Mean Loss: 0.0641
	Validation AP@.50:.05:.95: 0.240
	Validation AP@.5: 0.540
	Validation AP@.75: 0.165
	Validation Recall@.50:.05:.95: 0.356
	Validation Recall@.5: 0.685
	Validation Recall@.75: 0.315
	Validation ACD: 0.704
	Validation AAD: 0.667
	Best Validation AP@.50:.05:.95: 0.267

Epoch 100/100; Learning rate: 5e-05


Evaluating batches: 100%|██████████| 9/9 [00:03<00:00,  2.56it/s]


Time: 0:00:24; Epoch 100 Summary: 
	Train Mean Loss: 0.0643
	Validation AP@.50:.05:.95: 0.239
	Validation AP@.5: 0.537
	Validation AP@.75: 0.162
	Validation Recall@.50:.05:.95: 0.355
	Validation Recall@.5: 0.684
	Validation Recall@.75: 0.312
	Validation ACD: 0.706
	Validation AAD: 0.670
	Best Validation AP@.50:.05:.95: 0.267
